In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_bayesimp as file

In [2]:
# Args setup
ntrial = 50
n = 1000
ntest = 100
d = 5
noise = 0.5
niter = 1000
optimise_mu = [False, False, True]
exact = [False, True, True]
mc_samples = n
kernel = "gaussian"

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

2024-09-30 21:24:02,320 - distributed.scheduler - ERROR - Couldn't gather keys: {'main-80bbe079c186b6d651d14b3e79912e05': 'processing'}


In [4]:
# Submitting jobs
futures = []
for i in range(len(optimise_mu)):           
    for seed in range(ntrial):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                             optimise_mu = optimise_mu[i], exact = exact[i],
                             mc_samples = mc_samples, kernel = kernel,
                             niter = niter)
            futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-80bbe079c186b6d651d14b3e79912e05>,
 <Future: pending, key: main-01a90cf9ee5ac9b75993c44383b02157>,
 <Future: pending, key: main-0749cb677c3ce6722d905d5d0d135a33>,
 <Future: pending, key: main-569ef8c3b35954b2d64395a9e29e47e4>,
 <Future: pending, key: main-281b3cc705ca6f62ba2f910242ccf607>,
 <Future: pending, key: main-a38b8f66b8de5bd44e93bf58e6ba5e05>,
 <Future: pending, key: main-501c02a40def6daac0d2faba65393291>,
 <Future: pending, key: main-8957dc5f8affac6d0558bfb0e03b5c5e>,
 <Future: pending, key: main-d1ab5e3ded732d53b2d4792215876f6d>,
 <Future: pending, key: main-d00748aaca22924526b41a74b2f97219>,
 <Future: pending, key: main-399dc419c30c31657b259895ca187a3b>,
 <Future: pending, key: main-53aa26b4c22929a59d97afab56427592>,
 <Future: pending, key: main-b86f95856ec60c77bb2d72921dca9bde>,
 <Future: pending, key: main-7fc8deaed924f80d4e672371037996d1>,
 <Future: pending, key: main-cae02bd61afddd061134c435e80db03d>,
 <Future: pending, key: main-4b370003a7b

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "ablation_bayesimp_ntrial={0}_n={1}_d={2}_noise={3}_kernel={4}.pt".format(ntrial,n,d,noise,kernel))